In [63]:
library(tidyverse)
library(janitor)
library(magrittr)

In [64]:
mydata <- read_csv('fake claims data _4_.csv')

mydata <- clean_names(mydata) ##convert 'stay type' header to stay_type

#converting admit and discharge columns to a datetime format and creating a stay_length column
mydata %<>%
    mutate(date_of_admit= as.Date(date_of_admit, format= "%m/%d/%Y"), 
          date_of_discharge= as.Date(date_of_discharge, format= "%m/%d/%Y"),
          stay_length = date_of_discharge - date_of_admit)

Parsed with column specification:
cols(
  bene_id = col_integer(),
  claim_id = col_integer(),
  date_of_admit = col_character(),
  date_of_discharge = col_character(),
  `stay type` = col_character()
)


In [65]:
#creating new table of days since most recent claim for given beneficiary
new_claims <- mydata %>%
    arrange(bene_id, date_of_discharge) %>%
    group_by(bene_id) %>%
    mutate(days_since_previous_discharge = date_of_admit - lag(date_of_discharge)) %>%
    mutate(days_since_previous_discharge = as.integer(days_since_previous_discharge)) %>%
    ungroup() %>%
    select(claim_id, days_since_previous_discharge)

In [66]:
#join table above to our original dataframe
mydata <- mydata %>%
    left_join(new_claims, by = "claim_id")
head(mydata)

bene_id,claim_id,date_of_admit,date_of_discharge,stay_type,stay_length,days_since_previous_discharge
80310,10165,2015-04-18,2015-04-19,short,1 days,NA
83137,10126,2015-03-24,2015-03-24,ed,0 days,2
88682,10269,2015-07-03,2015-07-11,short,8 days,94
89261,10109,2015-03-13,2015-03-22,snf,9 days,NA
82200,10241,2015-06-10,2015-06-10,ed,0 days,24
83120,10446,2015-11-09,2015-11-14,short,5 days,218


In [67]:
#creating flags for new claims starting within x days of previous claim end for each beneficiary
#if_else may be a more performance-conscious choice than case_when when dealing with a larger dataset
mydata <- mydata%>%
    mutate(claim_flag = case_when(
                (days_since_previous_discharge <= 1) ~ "Within 1 day",
                (1 < days_since_previous_discharge & days_since_previous_discharge <= 30) ~ "Within 30 days",
                (31 <= days_since_previous_discharge & days_since_previous_discharge <= 180) ~ "Within 180 days",
                (days_since_previous_discharge > 180) ~ "More than 180 days"))

In [68]:
mydata <- mydata %>%
    arrange(bene_id, date_of_admit)

mydata

bene_id,claim_id,date_of_admit,date_of_discharge,stay_type,stay_length,days_since_previous_discharge,claim_flag
80006,10049,2015-02-01,2015-02-01,ed,0 days,NA,NA
80043,10131,2015-03-26,2015-03-26,ed,0 days,NA,NA
80043,10371,2015-09-17,2015-09-25,snf,8 days,175,Within 180 days
80043,10448,2015-11-10,2015-11-12,snf,2 days,46,Within 180 days
80161,10384,2015-09-26,2015-10-04,short,8 days,NA,NA
80238,10187,2015-05-06,2015-05-06,ed,0 days,NA,NA
80238,10188,2015-05-07,2015-05-13,snf,6 days,1,Within 1 day
80238,10438,2015-11-04,2015-11-06,short,2 days,175,Within 180 days
80238,10471,2015-11-27,2015-12-06,snf,9 days,21,Within 30 days
80238,10492,2015-12-25,2015-12-25,ed,0 days,19,Within 30 days


In [69]:
#can filter simply (e.g. by beneficiary)

mydata %>% filter(bene_id == 80043)

bene_id,claim_id,date_of_admit,date_of_discharge,stay_type,stay_length,days_since_previous_discharge,claim_flag
80043,10131,2015-03-26,2015-03-26,ed,0 days,NA,NA
80043,10371,2015-09-17,2015-09-25,snf,8 days,175,Within 180 days
80043,10448,2015-11-10,2015-11-12,snf,2 days,46,Within 180 days
